# Convert skymaps to poly coords

## Read raw skymap files

In [ ]:
import pickle
from pathlib import Path
import math

In [ ]:
home_dir = "/sdf/home/o/olynn/"
raw_skymaps_dir = Path(home_dir, "skymap-to-poly-coords", "tests", "data", "raw-skymaps")

In [ ]:
skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"
with open(skymap_path, "rb") as f:
    lsst_skymap = pickle.load(f)
lsst_skymap

## Rings sky map pixelization
*From the RingsSkyMap docstring in lsst.skymap:*

We divide the sphere into N rings of Declination, plus the two polar
caps, which sets the size of the individual tracts.  The rings are
divided in RA into an integral number of tracts of this size; this
division is made at the Declination closest to zero so as to ensure
full overlap.

Rings are numbered in the rings from south to north. The south pole cap is
``tract=0``, then the tract at ``raStart`` in the southernmost ring is
``tract=1``. Numbering continues (in the positive RA direction) around that
ring and then continues in the same fashion with the next ring north, and
so on until all reaching the north pole cap, which is
``tract=len(skymap) - 1``.

However, ``version=0`` had a bug in the numbering of the tracts: the first
and last tracts in the first (southernmost) ring were identical, and the
first tract in the last (northernmost) ring was missing. When using
``version=0``, these tracts remain missing in order to preserve the
numbering scheme.

## Get poly coords

In [ ]:
skymap = lsst_skymap

In [ ]:
#print("Size of a ring in Declination (radians):", skymap._ringSize)
print("Number of tracts:", skymap._numTracts)
print("Number of rings:", skymap.config.numRings)
print("Starting center RA for each ring (degrees):", skymap.config.raStart)

In [ ]:
", ".join(str(rN) for rN in skymap._ringNums)  # Number of tracts for each ring

In [ ]:
ring_bounds = []
print(f"{len(skymap._ringNums)} rings")
for i in range(skymap.config.numRings + 1):
        startDec = skymap._ringSize*(i + 0.5) - 0.5*math.pi
        stopDec = startDec + skymap._ringSize
        if i < skymap.config.numRings:
            print(f"Ring {i}: {startDec:.3f} to {stopDec:.3f} radians")

## Plotting

In [ ]:
# ring_size_radians = math.pi / (skymap.config.numRings + 1)  # Size of a ring in Declination (radians)
# ring_size_degrees = ring_size_radians * (180 / math.pi)
# print("Size of a ring in Declination (degrees):", ring_size_degrees)

#height = skymap._ringSize * (180 / math.pi) * u.deg  # Convert radians to degrees

In [ ]:
import astropy.units as u 
import matplotlib.pyplot as plt
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.visualization.wcsaxes import Quadrangle
from astropy.coordinates import Angle, SkyCoord
from mocpy import WCS, MOC

In [ ]:
kwargs = {
    "figsize": (9, 5),
}
projection = "MOL"
fig = plt.figure(**kwargs)
frame_class = EllipticalFrame
fov = (320 * u.deg, 160 * u.deg)
center = SkyCoord(0, 0, unit="deg", frame="icrs")
wcs = WCS(
    fig,
    fov=fov,
    center=center,
    coordsys="icrs",
    rotation=Angle(0, u.degree),
    projection=projection,
).w

ax = fig.add_subplot(1, 1, 1, projection=wcs, frame_class=frame_class)

# anchor = (0.0 * u.deg, 0.0 * u.deg)
# height = 60.0 * u.deg
# width = 120.0 * u.deg

# quad = Quadrangle(
#     anchor=anchor,
#     height=height,
#     width=width,
#     edgecolor="red",
#     facecolor="orange",
#     alpha=0.5
# )

# plt.grid()
# plt.ylabel("Dec")
# plt.xlabel("RA")
# plt.title("")
# plt.show()
